In [1]:
from google import genai
from google.genai import types
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.environ['GEMINI_API_KEY'],
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)

def call_gemini(system_message, user_message, response_schema = None, model="gemini-2.5-flash-lite"):
    if response_schema:
        response = client.chat.completions.parse(
            model = model,
            messages = [
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            temperature = 0.65,
            extra_body={
                
            },
            response_format = response_schema if response_schema else None
        ).choices[0].message.parsed    
        return response
    else:
        response = client.chat.completions.create(
            model = model,
            messages = [
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            temperature = 0.65,
            extra_body={
                
            }
        ).choices[0].message.content    
        return response

In [39]:
import json
import numpy as np
with open('similar_question_data.json','r') as f:
    dataset = json.load(f)

In [40]:
dataset = {item['question_id']: item for item in dataset}

In [29]:
with open('generated_solutions_w_similar.json','r') as f:
    generated_solutions_w_similar = json.load(f)
with open('generated_solutions_wo_similar.json','r') as f:
    generated_solutions_wo_similar = json.load(f)
    
generated_solutions_w_similar = {item['question_id']: item for item in generated_solutions_w_similar}
generated_solutions_wo_similar = {item['question_id']: item for item in generated_solutions_wo_similar}

In [ ]:
it_to_delete = []
for ques_id, data in dataset.items():
    if generated_solutions_w_similar.get(ques_id):
        data['solution_generated_with_similar'] = generated_solutions_w_similar.get(ques_id)['generated_solution']
        data['solution_generated_without_similar'] = generated_solutions_wo_similar.get(ques_id)['generated_solution']
    else:
        it_to_delete.append(ques_id)
[dataset.pop(x) for x in it_to_delete]

it_to_delete

In [4]:
dataset = np.asarray(dataset)

In [1]:
from datatypes import Solution, GeneratedSolution

In [62]:
np.random.choice(dataset,1,replace=False).tolist()

NameError: name 'np' is not defined

In [ ]:
from prompts import 
from datatypes import RelevanceSimilarity, RelevanceAlignment, EvaluationReport

In [ ]:
ques1 = dataset[325]
print(ques1['question_text'])
print(ques1['similar_questions'])

In [ ]:
formatted_similar_questions = "\n\n".join([f"### QUESTION {idx + 1}\n- {sq['similar_question_text']}" for idx,sq in enumerate(ques1['similar_questions'])])
formatted_system_prompt = relevance_similarity_system_prompt.format(subject = ques1['subject'])
formatted_user_prompt = relevance_user_prompt.format(main_question = ques1['question_text'], similar_questions = formatted_similar_questions)
print(call_gemini(formatted_system_prompt, formatted_user_prompt, RelevanceSimilarity, model="gemini-2.5-flash"))

In [ ]:
formatted_similar_questions = "\n\n".join([f"### QUESTION {idx + 1}\n- {sq['similar_question_text']}\n### SOLUTION APPROACH {idx + 1}\n- {sq['summarized_solution_approach']}" for idx,sq in enumerate(ques1['similar_questions'])])
formatted_system_prompt = relevance_alignment_system_prompt.format(subject = ques1['subject'])
formatted_user_prompt = relevance_user_prompt.format(main_question = ques1['question_text'], similar_questions = formatted_similar_questions)
print(call_gemini(formatted_system_prompt, formatted_user_prompt, EvaluationReport, model="gemini-2.5-flash"))

In [10]:
import json
with open('generated_solutions_wo_similar.json','r') as f:
    solutions = json.load(f)
with open('generated_solutions_w_similar.json','r') as f:
    solutions_with_similar = json.load(f)

with open('similar_question_data.json','r') as f:
    dataset = json.load(f)

In [ ]:
print(solutions[0]['generated_solution'])

In [ ]:
print(solutions_with_similar[0]['generated_solution'])